# Data Analysis - Smart Sambil

Este Notebook proporciona vistas SQL que devuelven datos interesantes de la base de datos del Sambil. También muestra los datos en tablas y gráficos. Puedes ver la definición de estas vistas y demás scripts de la base de datos en el archivo **important.sql**.

Antes de empezar recuerda restaurar la base de datos suministrada (**sambil_db.sql**).

Las tablas y gráficos se generan con consultas a la base de datos; si actualizas su datos puedes volver a ejecutar los scripts y ver los datos refejados.

Importemos las librerías que serán necesarias.

In [ ]:
import psycopg2
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

Establezcamos conexión con la base de datos (utiliza tus credenciales) e instanciemos un cursor.

In [ ]:
conn = psycopg2.connect(host='localhost', user='postgres', password='', dbname='')
cursor = conn.cursor()

Ya podemos realizar consultas.

### Porcentaje de ventas con dirección MAC (smartphone) contra ventas sin dirección MAC (sin smartphone)

In [ ]:
# Construimos la consulta
sql_query = 'SELECT * FROM ventas_totales_vs_mac;'

# Hacemos que pandas lea la consulta
df = pd.read_sql(sql_query, conn)

porcentajes = [df["Ventas MAC"][0], df["Ventas sin MAC"][0]]

plt.pie(porcentajes, labels=["Ventas MAC","Ventas sin MAC"], autopct="%0.1f%%")

# Mostrar tabla
df

### Top 5 de personas que más entran al Sambil

In [ ]:
# Construimos la consulta
sql_query = 'SELECT * FROM top_cinco_mac_entrada;'

# Hacemos que pandas lea la consulta
df = pd.read_sql(sql_query, conn)

height = df["Cantidad de entradas"]
bars = df["Nombre"] + " - " + df["Direccion MAC"]
y_pos = np.arange(len(bars))
 
plt.barh(y_pos, height)
 
plt.yticks(y_pos, bars)

# Mostrar tabla
df

### Reporte de ventas de las tiendas en forma descendente

In [ ]:
# Construimos la consulta
sql_query = 'SELECT * FROM ventas_mes_actual_por_tienda;'

# Hacemos que pandas lea la consulta
df = pd.read_sql(sql_query, conn)

df.plot(kind='bar', x='Tienda', y='Venta mensual')

# Mostrar tabla
df

### Reporte de visitas de las tiendas

In [ ]:
# Construimos la consulta
sql_query = 'SELECT * FROM cantidad_de_visitas_tienda_mes_actual;'

# Hacemos que pandas lea la consulta
df = pd.read_sql(sql_query, conn)

height = df["Cantidad de visitas"]
bars = df["Tienda"]
y_pos = np.arange(len(bars))
 
plt.barh(y_pos, height)
 
plt.yticks(y_pos, bars)

# Mostrar tabla
df

### Porcentaje de personas que se sientan en las mesas

In [ ]:
# Construimos la consulta
sql_query = 'SELECT * FROM entradas_vs_sentados;'

# Hacemos que pandas lea la consulta
df = pd.read_sql(sql_query, conn)

porcentajes = [df["Num. de sentados"][0], df["Entradas"][0] - df["Num. de sentados"][0]]

plt.pie(porcentajes, labels=["Se sientan","No se sientan"], autopct="%0.1f%%")

# Mostrar tabla
df

## Más gráficos

### Gráfico de dispersión  de ventas contra visitas por cada tienda

In [ ]:
# Construimos la consulta
sql_query = 'select * from ventas_mes_actual_por_tienda natural join cantidad_de_visitas_tienda_mes_actual;'

df = pd.read_sql(sql_query, conn)

types = df["Tienda"]
x_coords = df["Cantidad de visitas"]
y_coords = df["Venta mensual"]

for i,type in enumerate(types):
    x = x_coords[i]
    y = y_coords[i]
    plt.scatter(x, y, color='red')
    plt.text(x+0.3, y+0.3, type, fontsize=9)
plt.yticks(df["Venta mensual"])
print("Eje Y en millones")
plt.show()

### Gráfico de calor de las ubicaciones del centro comercial

A lo largo del plano del centro comercial se distribuyen nodos en una matriz cuadrada. Estos nodos corresponden a los de la base de datos. **Puedes interactuar con este gráfico**; cambiando el método para la interpolación. Cambia de método entre los proporcionados para ver el resultado. 

In [ ]:
import matplotlib.image as image

img = image.imread('../plano.png')

# Construimos la consulta
sql_query = 'select count(id_act) as "cantidad" from act_nodo group by id_nodo order by id_nodo asc;'

df = pd.read_sql(sql_query, conn)
r = []
i = 0
m = []
for index, row in df.iterrows():
    r.append(row['cantidad'])
    if(i == 4):
        m.append(r)
        r = []
        i = 0
        continue
    i += 1
    
m = np.asmatrix(m)

data = m

fig = plt.figure()
ax = fig.add_subplot(111)

# Lista de métodos
metodos_interpolacion = {
    1: 'nearest',   # Recomendado
    2: 'catrom',
    3: 'gaussian',
    4: 'mitchell',
    5: 'sinc',      # Recomendado
    6: 'lanczos'    # Recomendado
}

# Para cambiar el metodo, cambia el indice de metodos_interpolacion
cax = ax.matshow(data, cmap='plasma', interpolation=metodos_interpolacion[6])
fig.colorbar(cax)

print("La barra de la derecha representa la cantidad de visitas con su correspondiente color en la escala")

plt.imshow(img, alpha=0.4, aspect='auto', extent=[-0.5,4.5,3.5,-0.5])
plt.show()

### [IA] Cantidad de hombres y mujeres que entran al centro comercial

In [ ]:
sql_query = 'select * from hombres_vs_mujeres';

df = pd.read_sql(sql_query, conn)

plt.pie([df['Hombres'][0], df['Mujeres'][0]], labels=["Hombres","Mujeres"], autopct="%0.1f%%")
print("[97% de precision]")
plt.show()

### [IA] Gráfico de barras de cantidad de hombres (azul) y mujeres (rojo) por edad

In [ ]:
sql_query_hombres = 'select * from cantidad_hombres_por_edad;'
sql_query_mujeres = 'select * from cantidad_mujeres_por_edad;'

df_h = pd.read_sql(sql_query_hombres, conn)
df_m = pd.read_sql(sql_query_mujeres, conn)

fig, axes = plt.subplots(ncols=2, sharey=True)
axes[0].barh(df_h['Edad'], df_h['Cantidad'], align='center', color='blue')
axes[1].barh(df_m['Edad'], df_m['Cantidad'], align='center', color='red')
axes[0].invert_xaxis()
plt.show()